In [1]:
import json
from collections import Counter
import pandas as pd

In [11]:
# Read File

file_name = "MELD_train_efr.json"
with open(file_name, 'r') as file:
        dataset = json.load(file)

In [12]:
# Number of episodes in dataset

N = len(dataset)
print(N)

4000


In [13]:
# Sample Episode

dataset[0]

{'episode': 'utterance_0',
 'speakers': ['Chandler',
  'The Interviewer',
  'Chandler',
  'The Interviewer',
  'Chandler'],
 'emotions': ['neutral', 'neutral', 'neutral', 'neutral', 'surprise'],
 'utterances': ["also I was the point person on my company's transition from the KL-5 to GR-6 system.",
  "You must've had your hands full.",
  'That I did. That I did.',
  "So let's talk a little bit about your duties.",
  'My duties?  All right.'],
 'triggers': [0.0, 0.0, 0.0, 1.0, 0.0]}

In [14]:
# Reading the data

episodes_list = []
speakers_list = []
utterances_list = []
triggers_list = []
emotions_list = []

for i in range(len(dataset)):
        episodes_list.append(dataset[i]['episode'])
        speakers_list.append(dataset[i]['speakers'])
        utterances_list.append(dataset[i]['utterances'])
        triggers_list.append(dataset[i]['triggers'])
        emotions_list.append(dataset[i]['emotions'])  
    

In [6]:
headings = ["Dialogue_Id", "Speaker", "Emotion_name", "Utterance", "Annotate(0/1)"]

In [7]:
data = [[],[],[],[],[]]

In [15]:
data = [[],[],[],[],[]]
prev_first_sen = None
d_id = -1
for i in range(N):
    if prev_first_sen!=utterances_list[i][0]:
        d_id+=1
        prev_first_sen = utterances_list[i][0]
        
    for j in range(len(utterances_list[i])):
        data[0].append(d_id)
        data[1].append(speakers_list[i][j])
        data[2].append(emotions_list[i][j])
        data[3].append(utterances_list[i][j])
        data[4].append(triggers_list[i][j])
        
    data[0].append("")
    data[1].append("")
    data[2].append("")
    data[3].append("")
    data[4].append("")
                       

In [16]:
print(len(data[0]))

42948


In [23]:
data

[[0,
  0,
  0,
  0,
  '',
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  '',
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  '',
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  '',
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  '',
  1,
  1,
  1,
  '',
  1,
  1,
  1,
  1,
  1,
  '',
  1,
  1,
  1,
  1,
  1,
  1,
  '',
  2,
  2,
  2,
  '',
  2,
  2,
  2,
  2,
  '',
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  '',
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  '',
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  '',
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  '',
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  '',
  3,
  3,
  3,
  3,
  '',
  4,
  4,
  4,
  '',
  4,
  4,
  4,
  4,
  4,
  '',
  4,
  4,
  4,
  4,
  4,
  4,
  '',
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  '',
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  '',
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,

In [24]:
import nlp_utils as nu

In [25]:
nu.preprocess_text(utterances_list[0][0])

'or or we could go to the bank close our accounts and cut them off at the source'

In [14]:
df = pd.DataFrame(data)
df = df.transpose()
df.columns = headings
df.to_csv("MELD_train_dev_my_old.csv", index=False)

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

In [ ]:
def get_sen_embed(utt):
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():

        # Define a new example sentence with multiple meanings of the word "bank"
        text = utt

        # Add the special tokens.
        marked_text = "[CLS] " + text + " [SEP]"

        # Split the sentence into tokens.
        tokenized_text = tokenizer.tokenize(marked_text)

        # Map the token strings to their vocabulary indeces.
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

        # Mark each of the 22 tokens as belonging to sentence "1".
        segments_ids = [1] * len(tokenized_text)

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]
        # print(outputs)

    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(hidden_states, dim=0)

    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    # Stores the token vectors, with shape [22 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)
        
        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)

    # `hidden_states` has shape [13 x 1 x 22 x 768]

    # `token_vecs` is a tensor with shape [22 x 768]
    token_vecs = hidden_states[-2][0]

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    return sentence_embedding

In [ ]:
ans_dict = {}

In [ ]:
for i in range(N):
    for j in range(len(utterances_list[i])):
        pp_utt = nu.preprocess_text(utterances_list[i][j])
        utt_emb = get_sen_embed(pp_utt)
        ans_dict[pp_utt] = utt_emb

In [ ]:
len(ans_dict)

In [ ]:
ans_dict